In [49]:
import pandas as pd
from dateutil import parser
import requests
import sqlite3
import datetime

In [50]:
conn = sqlite3.connect('database.sqlite')
c = conn.cursor()

In [51]:
c.execute("""SELECT * FROM Matches;""")
matches = pd.DataFrame(c.fetchall())
matches.columns = [x[0] for x in c.description]
matches.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A


In [52]:
gameDate = []

for i in range(len(matches['Date'])):
    dt = parser.parse(matches['Date'][i])
    gameDate.append(dt)

In [53]:
matches['gamedate'] = gameDate

In [54]:
matches_2011 = matches[matches["Season"] == 2011]
matches_2011.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,gamedate
1091,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,2012-03-31
1092,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,2011-12-11
1093,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,2011-08-13
1094,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,2011-11-27
1095,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,2012-02-18


In [55]:
matches_2011 = matches_2011.reset_index()

In [56]:
matches_2011.head()

,index,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,gamedate
0,1091,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,2012-03-31
1,1092,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,2011-12-11
2,1093,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,2011-08-13
3,1094,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,2011-11-27
4,1095,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,2012-02-18


In [57]:
berlin_lat = '52.520008'
berlin_long = '13.404954'

In [58]:
dates = list(set(matches_2011['gamedate']))

In [60]:
timestamp = []
for date in range(len(dates)):
    stamp = str(matches_2011['gamedate'][date].timestamp())
    timestamp.append(stamp[:-2])


In [61]:
len(timestamp)

165

In [62]:
api_key = '' #GET YOUR OWN YOU MOOCH

In [15]:
resp = requests.get('https://api.darksky.net/forecast/' + api_key + '/' + berlin_lat + ',' + berlin_long + ',' + timestamp[0] + '?exclude=flags,hourly,uvIndex,windSpeed')
resp.json()['currently']

{'time': 1333152000,
 'summary': 'Breezy and Mostly Cloudy',
 'icon': 'wind',
 'precipIntensity': 0,
 'precipProbability': 0,
 'temperature': 46.44,
 'apparentTemperature': 39.02,
 'dewPoint': 39.54,
 'humidity': 0.77,
 'windSpeed': 19.55,
 'windGust': 35.66,
 'windBearing': 281,
 'cloudCover': 0.75,
 'uvIndex': 0,
 'visibility': 6.22}

In [28]:
dark_sky = []

def get_rain(lat, long, time, api_key):
    for days in time:
        resp = requests.get('https://api.darksky.net/forecast/' + api_key + '/' + lat + ',' + long + ',' + days + '?exclude=flags,hourly,alerts')
        dark_sky.append(resp.json())
    return dark_sky

In [29]:
weather = get_rain(berlin_lat, berlin_long, timestamp, api_key)
weather

[{'latitude': 52.520008,
  'longitude': 13.404954,
  'timezone': 'Europe/Berlin',
  'currently': {'time': 1333152000,
   'summary': 'Breezy and Mostly Cloudy',
   'icon': 'wind',
   'precipIntensity': 0,
   'precipProbability': 0,
   'temperature': 46.44,
   'apparentTemperature': 39.02,
   'dewPoint': 39.54,
   'humidity': 0.77,
   'windSpeed': 19.55,
   'windGust': 35.66,
   'windBearing': 281,
   'cloudCover': 0.75,
   'uvIndex': 0,
   'visibility': 6.22},
  'daily': {'data': [{'time': 1333144800,
     'summary': 'Breezy until afternoon and mostly cloudy throughout the day.',
     'icon': 'wind',
     'sunriseTime': 1333168981,
     'sunsetTime': 1333215601,
     'moonPhase': 0.27,
     'precipIntensity': 0.0003,
     'precipIntensityMax': 0.0029,
     'precipIntensityMaxTime': 1333180800,
     'precipProbability': 0.56,
     'precipType': 'rain',
     'temperatureHigh': 44.62,
     'temperatureHighTime': 1333188000,
     'temperatureLow': 30.12,
     'temperatureLowTime': 133325640

In [37]:
weather[1]['daily']['data'][0]

{'time': 1323558000,
 'summary': 'Partly cloudy until afternoon.',
 'icon': 'partly-cloudy-day',
 'sunriseTime': 1323587293,
 'sunsetTime': 1323615220,
 'moonPhase': 0.52,
 'precipIntensity': 0,
 'precipIntensityMax': 0,
 'precipProbability': 0,
 'temperatureHigh': 41.03,
 'temperatureHighTime': 1323604800,
 'temperatureLow': 31.04,
 'temperatureLowTime': 1323655200,
 'apparentTemperatureHigh': 34.91,
 'apparentTemperatureHighTime': 1323604800,
 'apparentTemperatureLow': 23.31,
 'apparentTemperatureLowTime': 1323658800,
 'dewPoint': 29.26,
 'humidity': 0.77,
 'windSpeed': 8.48,
 'windGust': 14.99,
 'windGustTime': 1323558000,
 'windBearing': 220,
 'cloudCover': 0.53,
 'uvIndex': 0,
 'uvIndexTime': 1323558000,
 'visibility': 6.21,
 'temperatureMin': 33.03,
 'temperatureMinTime': 1323640800,
 'temperatureMax': 41.03,
 'temperatureMaxTime': 1323604800,
 'apparentTemperatureMin': 25.41,
 'apparentTemperatureMinTime': 1323640800,
 'apparentTemperatureMax': 34.91,
 'apparentTemperatureMaxTim

In [64]:
ds_df = pd.read_csv('weather.csv')

In [65]:
ds_df[ds_df['summary'] == 'Drizzle']

,Unnamed: 0,apparentTemperature,cloudCover,dewPoint,humidity,icon,precipIntensity,precipProbability,precipType,summary,temperature,time,uvIndex,visibility,windBearing,windGust,windSpeed
8,8,61.74,0.86,58.77,0.90,rain,0.0169,0.48,rain,Drizzle,61.63,1316304000,0,6.22,140.0,12.01,5.74
26,26,61.74,0.86,58.77,0.90,rain,0.0169,0.48,rain,Drizzle,61.63,1316304000,0,6.22,140.0,12.01,5.74
157,157,34.58,0.86,37.40,0.82,rain,0.0073,0.48,rain,Drizzle,42.62,1323388800,0,6.22,200.0,17.25,17.25


In [66]:
df_rain = pd.DataFrame()

In [67]:
df_rain['rainy'] = ds_df['icon']

In [68]:
df_rain['date'] = ds_df['time']

In [69]:
datetime.datetime.utcfromtimestamp(ds_df['time'][1]).strftime('%Y-%m-%d')

'2011-12-11'

In [70]:
for day in range(len(ds_df['time'])):
    ds_df['time'][day] = datetime.datetime.utcfromtimestamp(ds_df['time'][day]).strftime('%Y-%m-%d')

/Users/robblatt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/robblatt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [71]:
ds_df

,Unnamed: 0,apparentTemperature,cloudCover,dewPoint,humidity,icon,precipIntensity,precipProbability,precipType,summary,temperature,time,uvIndex,visibility,windBearing,windGust,windSpeed
0,0,39.02,0.75,39.54,0.77,wind,0.0000,0.00,NaN,Breezy and Mostly Cloudy,46.44,2012-03-31,0,6.22,281.0,35.66,19.55
1,1,28.68,0.61,30.19,0.75,partly-cloudy-night,0.0000,0.00,NaN,Mostly Cloudy,37.22,2011-12-11,0,6.22,251.0,14.99,13.82
2,2,60.14,0.75,58.19,0.94,partly-cloudy-night,0.0000,0.00,NaN,Mostly Cloudy,60.03,2011-08-13,0,6.21,180.0,4.00,2.27
3,3,41.85,0.75,40.02,0.74,partly-cloudy-night,0.0000,0.00,NaN,Mostly Cloudy,47.81,2011-11-27,0,6.22,230.0,14.96,14.96
4,4,32.88,0.75,34.06,0.88,partly-cloudy-night,0.0000,0.00,NaN,Mostly Cloudy,37.22,2012-02-18,0,6.22,270.0,5.74,5.41
5,5,28.23,0.30,33.23,0.85,partly-cloudy-night,0.0000,0.00,NaN,Partly Cloudy,37.43,2012-01-20,0,6.22,260.0,16.00,16.00
6,6,6.03,0.30,3.25,0.88,partly-cloudy-night,0.0000,0.00,NaN,Partly Cloudy,6.03,2012-02-04,0,6.22,260.0,2.29,2.29
7,7,44.57,NaN,37.40,0.76,clear-night,0.0000,0.00,NaN,Clear,44.57,2012-04-21,0,6.21,170.0,NaN,1.02
8,8,61.74,0.86,58.77,0.90,rain,0.0169,0.48,rain,Drizzle,61.63,2011-09-18,0,6.22,140.0,12.01,5.74
9,9,31.37,NaN,32.80,0.85,clear-night,0.0000,0.00,NaN,Clear,36.83,2011-10-23,0,6.21,120.0,8.99,6.91


In [72]:
rain = pd.DataFrame()

In [73]:
rain['date'] = ds_df['time']

In [74]:
rain['raining'] = False

In [75]:
for i in range(len(ds_df['precipType'])):
    if ds_df['precipType'][i] == 'rain':
        rain.raining[i] = True
    else:
        rain.raining[i] = False

/Users/robblatt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/robblatt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [76]:
rain_days = list(rain.date[rain['raining'] == True])

In [77]:
rain_days

['2011-09-18',
 '2011-12-17',
 '2011-09-18',
 '2012-05-06',
 '2012-05-06',
 '2012-05-06',
 '2011-08-05',
 '2011-12-09']

In [185]:
matches_2011['Date'][2]

'2011-08-13'

In [202]:
rain_2011 = []
for i in range(len(matches_2011['Date'])):
    if matches_2011['Date'][i] in rain_days:
        rain_2011.append(True)
    else:
        rain_2011.append(False)

In [203]:
matches_2011['rain'] = rain_2011

In [205]:
matches_2011

,index,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,gamedate,rain
0,1091,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,2012-03-31,False
1,1092,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,2011-12-11,False
2,1093,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,2011-08-13,False
3,1094,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,2011-11-27,False
4,1095,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,2012-02-18,False
5,1096,1097,D1,2011,2012-01-20,M'gladbach,Bayern Munich,3,1,H,2012-01-20,False
6,1097,1098,D1,2011,2012-02-04,Hamburg,Bayern Munich,1,1,D,2012-02-04,False
7,1098,1099,D1,2011,2012-04-21,Werder Bremen,Bayern Munich,1,2,A,2012-04-21,False
8,1099,1100,D1,2011,2011-09-18,Schalke 04,Bayern Munich,0,2,A,2011-09-18,True
9,1100,1101,D1,2011,2011-10-23,Hannover,Bayern Munich,2,1,H,2011-10-23,False
